In [48]:
import glob
import numpy as np
import json
import re

<h4>Get All Lyrics:</h4>

In [89]:
files  = glob.glob('lyrics/**/*json')
l = []
for i in files: 
    with open(i, 'r') as f:
        d = json.load(f)
        l.append(d)
print(len(l))

1415


In [23]:
#files  = glob.glob('lyrics/Drake/*json')
#l = []
#for i in files: 
#    with open(i, 'r') as f:
#        d = json.load(f)
#        l.append(d)

<h4>Convert Lyrics to Text File to finetune GPT-2:</h4>

In [93]:
def get_input_output(lyric,artist,num_input_lines): 
    '''
    First Bar of Verse 1 is input, rest of song is output
    '''
    ind = lyric.find("[Verse 1: {}]".format(artist))
    if ind == -1:
         ind = lyric.find("[Verse 1]")
    bar0 = "\n".join(lyric[ind:].split("\n")[1:num_input_lines+1])
    rest = lyric[ind:].split("\n")[num_input_lines+1:]
    rest = "\n".join(rest)
    rest = re.sub("[\[].*?[\]]", "", rest)
    return bar0, rest

lyrics_io = []
for i in l:  
    artist = i['artist']
    lyric = i['songs'][0]['lyrics']
    bar0, rest = get_input_output(lyric,artist,4)
    d = {}
    d['input'] = bar0
    d['output'] = rest
    lyrics_io.append(d)


def get_finetune_text(lyrics_io,filename):
    dta = "<|endoftext|>".join("{0}|{1}".format(j['input'], j['output']) for j in lyrics_io)
    Path(filename).write_text(dta)
    print('File Written')
    
get_finetune_text(lyrics_io,"test.txt")

File Written


<h4>Download GPT-2 Model:</h4>

In [1]:
import gpt_2_simple as gpt2

#gpt2.download_gpt2()   # model is saved into current directory under /models/117M/

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


<h4>FineTune Model:</h4>

In [2]:
model_name = '117M'
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              'test.txt',
              model_name=model_name,
              steps=1)   # steps is max number of training steps

gpt2.generate(sess)

Loading checkpoint models/117M/model.ckpt
INFO:tensorflow:Restoring parameters from models/117M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:03<00:00,  3.33s/it]


dataset has 641646 tokens
Training...
[1 | 97.77] loss=3.51 avg=3.51
Saving checkpoint/run1/model-1
âînîs (reserved)

ġêtîrî

ġêtîrîlâî

Ó Ó Ó

É Ó

É Ó

E é é é é é é é é é é é é é é é é é é é é é é é é é é é é é é é é é

É é é é ééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééééé

In [ ]:
#'sess = gpt2.start_tf_sess()
#gpt2.load_gpt2(sess)

gpt2.generate(sess)

In [ ]:
gpt2.generate(sess, return_as_list=True)[0]